In [89]:
import logging
from collections import namedtuple
import random
from copy import deepcopy
from itertools import accumulate
from operator import xor


Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = None):
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        #self._rows = [0,1,3]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self):
        return self._k
    
    #apply the chosen move by removing num_objects from the row
    def nimming(self, ply: Nimply):
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


In [90]:
"""Pick always the minimum(maximum) possible number of the lowest row"""
def dumb_PCI(state: Nim):
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], -m[1])))

"""remove from a random row a random number of objects"""
def pure_random(state: Nim):
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

"""optimal strategy"""
def nim_sum(state: Nim):
    *_, result = accumulate(state.rows, xor)
    return result

def cook_status(state: Nim):
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

def optimal_startegy(state: Nim):
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]


In [91]:
import numpy as np
logging.getLogger().setLevel(logging.DEBUG)

alpha=0.15
random_factor=0.2  # 80% explore, 20% exploit
state_history = []
rewards = {}
RANDOM_THRESHOLD=0.9
TRAINING_MATCHES = 20000
TESTING_MATCHES = 100
NIM_SIZE = 8

In [92]:

def give_rewards(state: Nim):
    if not state:
        return 1
    active_rows=0
    for o in state.rows:
        if o>0:
            active_rows+=1
    if active_rows==1:
        return -1
    return 0

def recursive_function_for_rewards(state:Nim, possible_state: list, row: int):
    if row>=len(state.rows):
        poss_stat=tuple(possible_state)
        rewards[poss_stat]=np.random.uniform(low=1.0,high=0.1)
        return
    row+=1
    for i in range(state.rows[row-1]+1):
        possible_state2=deepcopy(possible_state)
        possible_state2.append(i)
        recursive_function_for_rewards(state,possible_state2,row)

def init_rewards(state: Nim):
    possible_state=[]
    recursive_function_for_rewards(state,possible_state, 0)

def choose_action(nim, allowedMoves, rewards):
        maxG = -10e15
        next_move = None
        randomN = np.random.random()
        if randomN < random_factor:
            # if random number below random factor, choose random action
            index = np.random.randint(0, len(allowedMoves))
            next_move = allowedMoves[index]
        else:
            # if exploiting, gather all possible actions and choose one with the highest G (reward)
            for action in allowedMoves:
                tmp = deepcopy(nim)
                tmp.nimming(action)
                new_state = tmp.rows
                if rewards[new_state] >= maxG:
                    next_move = action
                    maxG = rewards[new_state]
            
        return next_move
    
def allowed_moves(nim):
    return [
        (r, o) for r, c in enumerate(nim.rows) for o in range(1, c + 1)
    ]

def learn():
    global state_history
    global rewards
    global random_factor
    target = 0
    for prev, reward in reversed(state_history):
        rewards[tuple(prev)] = rewards[tuple(prev)] + alpha * (target - rewards[tuple(prev)])
        target +=reward
    state_history = []
    random_factor -= 10e-5  # decrease random factor each episode of play
        
def reinforcement_learning(nim):
    allowedMoves = allowed_moves(nim)
    action = choose_action(nim,allowedMoves, rewards)
    nim.nimming(action)
    rew = give_rewards(nim)
    state_history.append((nim.rows, rew))
    
    return action

def test(nim):
    allowedMoves = allowed_moves(nim)
    action = choose_action(nim,allowedMoves, rewards)
    nim.nimming(action)
    
    return action

    
nim = Nim(NIM_SIZE)
init_rewards(nim)
#print(f"pesi iniziali = {rewards}")

"""Training"""
for i in range(TRAINING_MATCHES):
    player = 0
    while nim:
        if player == 0:
            ply = reinforcement_learning(nim)
        else:
            if random.random()<=RANDOM_THRESHOLD:
                ply = optimal_startegy(nim)
            else:
                ply = pure_random(nim)
            nim.nimming(ply)
        
        player = 1 - player
    learn()
    winner = 1 - player
    nim = Nim(NIM_SIZE)
    
#print(f"pesi finali = {rewards}")

"""Evaluation of RL strategy"""
won = 0
for i in range(TESTING_MATCHES):
    player = 0
    while nim:
        if player == 0:
            ply = test(nim)
        else:
            ply = pure_random(nim)
            nim.nimming(ply)
        player = 1 - player
        
    if player == 1:
        won += 1
        
    nim = Nim(NIM_SIZE)
        
win_rate = won / TESTING_MATCHES
print(f"win rate = {win_rate}")

win rate = 0.86
